# Neighborhood Geocoding

In [85]:
import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup

In [86]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

html_data = BeautifulSoup(page.content, 'html.parser')

table = html_data.find("table")

In [87]:
neighborhoods = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in table.find("tbody").find_all("tr"):
    col = row.find_all("td")
    if str(col) != '[]':
        pcode = col[0].get_text("\\n", strip=True)
        borough = col[1].get_text("\\n", strip=True)
        neighborhood = col[2].get_text("\\n", strip=True)

        neighborhoods = neighborhoods.append({"PostalCode":pcode, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)

In [88]:
neighborhoods['Borough'].replace('Not assigned', np.nan, inplace=True)
neighborhoods.dropna(inplace=True)

### **Wiki appears to have cleaned the data and so these don't actually do anything.**

In [89]:
#neighborhoods.groupby(['PostalCode','Borough'], axis=1).agg(lambda x: ','.join(x.values))
#neighborhoods['Neighborhood'].replace('Not assigned', neighborhoods['Borough'], inplace=True)

In [90]:
neighborhoods.shape

(103, 3)

In [91]:
file_name ='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
geocodes = pd.read_csv(file_name)
geocodes.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [92]:
neighborhoods = neighborhoods.merge(geocodes, how='inner',on='PostalCode')

In [95]:
neighborhoods.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [96]:
neighborhoods.shape

(103, 5)